In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from anthropic import Anthropic

In [2]:
import gradio as gr

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

In [3]:
openai = OpenAI()
anthropic = Anthropic()

In [4]:
system_message = "You are a helpful assistant"


def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]

    completion = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

    return completion.choices[0].message.content

In [ ]:
message_gpt("Hello Whats Going On")

"Hello! I'm here to help you with any questions or information you might need. What's on your mind?"

In [6]:
def shout(text):
    return text.upper()

In [7]:
shout("Hello")

'HELLO'

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
gr.Interface(
    fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never"
).launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [10]:
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your Message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
)

In [11]:
view

Gradio Interface for: message_gpt
---------------------------------
inputs:
|-<gradio.components.textbox.Textbox object at 0x00000268F552E4E0>
outputs:
|-<gradio.components.textbox.Textbox object at 0x00000268F51F97F0>

In [12]:
view.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [13]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message")],
    outputs=[gr.Textbox(label="Response")],
    allow_flagging="never",
)

view.launch()

c:\Work\LLMs\.venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [14]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]

    stream = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages, stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [15]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your Message:")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never",
)

c:\Work\LLMs\.venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [16]:
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [20]:
def stream_claude(prompt):
    result = anthropic.messages.stream(
        model="claude-3-5-haiku-latest",
        max_tokens=100,
        temperature=0.7,
        system=system_message, 
        messages=[{'role':'user', 'content': prompt}]
    )
    response = "" 

    with result as stream: 
        for text in stream.text_stream: 
            response += text or "" 
            yield response 
            


In [21]:
view = gr.Interface(
    fn = stream_claude, 
    inputs=[gr.Textbox(label='Your Message:')], 
    outputs=[gr.Markdown(label='Response')], 
    allow_flagging='never'
)

c:\Work\LLMs\.venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [22]:
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [23]:
def stream_model(prompt, model):
    if model == "gpt":
        result = stream_gpt(prompt)
    elif model == "claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown Model")
    for chunk in result:
        yield chunk

In [24]:
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="Your Message:"),
        gr.Dropdown(["gpt", "claude"], label="Select Model"),
    ],
    outputs = [gr.Markdown(label='Response:')], 
    allow_flagging='never'
)

view.launch()

c:\Work\LLMs\.venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [25]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [26]:
system_prompt = """YOu are an assistant that analyzes the contents of a company website landing page and creates a short brochure about the company 
for prospective customers, investors and recruits, respond in markdown:""" 

In [27]:
def stream_brochure(company_name, url, model): 
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page: \n" 
    prompt += Website(url).get_contents() 
    if model == 'gpt': 
        result = stream_gpt(prompt) 
    elif model == 'claude': 
        result = stream_claude(prompt) 
    else: 
        raise ValueError('Unknown Model')
    
    for chunk in result: 
        yield chunk 

In [30]:
view = gr.Interface(fn = stream_brochure, inputs = [
    gr.Textbox(label='Company Name:'), 
    gr.Textbox(label='Landing Page URL:'), 
    gr.Dropdown(['gpt','claude'], label='Select Model'), 
], outputs=[gr.Markdown(label='Brochure:')], 
allow_flagging = 'never')

c:\Work\LLMs\.venv\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [31]:
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Work\LLMs\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Work\LLMs\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Work\LLMs\.venv\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Work\LLMs\.venv\Lib\site-packages\gradio\blocks.py", line 1601, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Work\LLMs\.venv\Lib\site-packages\gradio\utils.py", line 728, in async_iteration
    return await anext(iterator)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Work\LLMs\.venv\Lib\site-packages